In [29]:
import pandas as pd
movies = pd.read_csv('C:/Users/yoon/Desktop/NewFolder/project/ml-25m/movies.csv')
ratings = pd.read_csv('C:/Users/yoon/Desktop/NewFolder/project/ml-25m/ratings.csv')

ratings_m = ratings.groupby(['movieId'])['rating'].mean()
ratings_m = pd.DataFrame(ratings_m).reset_index()
movies_1 = pd.merge(movies, ratings_m, on=['movieId'])

ratings_c = ratings.groupby(['movieId']).size().reset_index(name='counts')
movies_1 = pd.merge(movies_1, ratings_c, on='movieId')
movies_1['released'] = movies_1.title.str.extract(r'(\d{4})')
                                        # movies_1 : rating과 released추가 

def fn1_movie_recommend():
    movies_4 = movies_1.dropna() # default : axis = 0, any = 'how'
    movies_4['released'] = movies_4['released'].astype('int64') # type변경을 안하면 에러발생.
    movies_4 = movies_4[(movies_4['released']>=1895) & (movies_4['released']<=2019)]
    movies_4['value'] = movies_4['rating']*4/5 + movies_4['counts']/movies_4['counts'].max()*4 + (movies_4['released']-movies_4['released'].min())/(movies_4['released'].max()-movies_4['released'].min())*2
    movies_4 = movies_4.sort_values('value',ascending=False, ignore_index=True)
    print(movies_4['title'].head(10))
                        # movies_4은 rating(4점 만점) + counts(4점 만점) + 연도조정치(2)를 합산한 값인 value 기준으로 정렬
        
def fn2_multiple_conditions(gen, year):
    movies_5 = movies_1.dropna()
    movies_5['released'] = movies_5['released'].astype('int64')
    movies_5 = movies_5[(movies_5['released']>=1895) & (movies_5['released']<=2019)]
    movies_5['value'] = movies_5['rating']*4/5 + movies_5['counts']/movies_5['counts'].max()*4 + (movies_5['released']-movies_5['released'].min())/(movies_5['released'].max()-movies_5['released'].min())*2
    movies_5 = movies_5.sort_values('value',ascending=False, ignore_index=True)
    movies_5 = movies_5.loc[movies_5['genres'].str.contains(gen, case=False) & (movies_5['released'] == year)]
    print(movies_5['title'].head(10))

def fn3_top_counts_movies():
    movies_2 = movies_1.sort_values('counts', ascending=False, ignore_index=True)
    print(movies_2['title'].head(10))
                # movies_2 : rating순 데이터프레임

def fn4_top_rating_movies():
    movies_3 = movies_1[movies_1['counts']>25]
    movies_3 = movies_3.sort_values('rating', ascending=False, ignore_index = True)
    print(movies_3['title'].head(10))

def fn5_real_time_search():
    from selenium import webdriver
    import requests, re, urllib
    import time
    dv = webdriver.Chrome('D:/bigdata/download/chromedriver_win32/chromedriver.exe')
    url = 'https://movie.naver.com/movie/point/af/list.naver'
    dv.get(url) # url에 해당하는 페이지가 open

    for i in range(1,12):
        page_btn = dv.find_element_by_css_selector('div.paging a:nth-child(' + str(i) + ') > span') # 이 부분이 해결되어야 함!!!
        page_btn.click()
        time.sleep(1)

        target = urllib.request.urlopen("https://movie.naver.com/movie/point/af/list.naver?&page="+str(i))
        contents = target.read().decode('utf-8')
        crude = re.findall(r'<span class="st_on".+?<a href="#"', contents, re.DOTALL)

        i = i + 1
        ready_to_print = " "
        for c in crude:
            ready = re.search(r'<br>(.+)\n\t\t\t\n\t\t\t\n\t\t\t\n\t\t\t\t\n\t\t\t\t\n\t\t\t\t\n\t\t\t\t<a href="#"', c)
            ready_to_print += ready.group(1)

        with open('D:/bigdata/note/project/movie_comments.txt', 'a', encoding='utf-8') as f:
            f.write(ready_to_print)

    #########################################################################################################################

    from konlpy.tag import Kkma
    kkma = Kkma(max_heap_size=1024)

    target = open('D:/bigdata/note/project/movie_comments.txt', encoding = 'utf-8').read()
    word_list = kkma.pos('%r' % target)
    word_list = [t[0] for t in word_list if t[1]=='NNG']
    text = ' '.join(word_list)

    from wordcloud import WordCloud # 클래스이므로 대문자
    from wordcloud import STOPWORDS
    import matplotlib.pyplot as plt # import matplotlib as plt로 하면 에러발생
    from PIL import Image
    import numpy as np
    img = Image.open('D:/bigdata/src/09_자연어처리/twitter-computer-black-icons-free-frame.png').convert('RGBA')
    mask = np.array(img)

    불용어 = set(['영화', '연기', '배우'])
    wc = WordCloud(background_color = 'white',
                  max_words=300,
                  font_path = 'C:/Windows/Fonts/한컴산뜻돋움/HanSantteutDotumBold.ttf',
                  relative_scaling = 0.5,
                  stopwords = 불용어,
                  mask=mask)

    wc.generate(text) # 워드 클라우드 생성
    plt.figure(figsize=(12,5))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off')
    
def fn9_end():
    pass
    
####################################################################################################
# 1,2 실행의 결과와 3 실행의 결과가 달라 break가 있는 경우와 없는 경우의 효과를 확인해볼 수 있음!
def main():
    
    while True:
        print("1:추천영화 | 2:다중조건 조회 | 3: 인기영화 | 4: 고평점 영화 | 5:실시간 트렌드(관객평) | 9:종료",sep='|')
        fn = int(input('메뉴선택 : '))
        if fn == 1:
            fn1_movie_recommend()
        elif fn == 2:
            try:
                year = int(input('조회를 희망하는 연도를 입력해주세요 : '))
                if year < 1895:
                    print('1895년 이후의 영화만 조회가 가능합니다')
                    break
                elif year > 2019:
                    print('2019년까지의 영화만 조회가 가능합니다')
                    break
#                break 오류가 없으면 break가 작동하고 if문 밖으로 빠져나감.
            except ValueError as e:
                print(e)
            except Exception as e:
                print(e)
            gen = input('장르를 선택해주세요 : ')
            fn2_multiple_conditions(gen, year)    
        
#                 if year >= 1895 and year <= 2019:
#                     movie_by_year(year)
#                 elif len(year) != 4:
#                     print("네 자리로 입력해주세요 ")
#                     break
        elif fn == 3:
            fn3_top_counts_movies()
        elif fn == 4:
            fn4_top_rating_movies()            
        elif fn == 5:
            fn5_real_time_search()
            break
        elif fn == 9:
            fn9_end()
            break

In [30]:
if __name__ == '__main__':
    main()

1:추천영화 | 2:다중조건 조회 | 3: 인기영화 | 4: 고평점 영화 | 5:실시간 트렌드(관객평) | 9:종료
메뉴선택 : 2
조회를 희망하는 연도를 입력해주세요 : 1995
장르를 선택해주세요 : comedy


<ipython-input-29-510877ae3500>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_5['released'] = movies_5['released'].astype('int64')


13                            Toy Story (1995)
862                            Clueless (1995)
991                          Get Shorty (1995)
1164            While You Were Sleeping (1995)
1165    Wallace & Gromit: A Close Shave (1995)
1301                     Batman Forever (1995)
1429            American President, The (1995)
4713                              Smoke (1995)
5177           Welcome to the Dollhouse (1995)
5353           Antonia's Line (Antonia) (1995)
Name: title, dtype: object
1:추천영화 | 2:다중조건 조회 | 3: 인기영화 | 4: 고평점 영화 | 5:실시간 트렌드(관객평) | 9:종료
메뉴선택 : 3
0                          Forrest Gump (1994)
1             Shawshank Redemption, The (1994)
2                          Pulp Fiction (1994)
3             Silence of the Lambs, The (1991)
4                           Matrix, The (1999)
5    Star Wars: Episode IV - A New Hope (1977)
6                         Jurassic Park (1993)
7                      Schindler's List (1993)
8                            Braveheart (1995)
9     